# JTWC bulletin parser into inpData

In [1]:
import feedparser

In [2]:
import numpy as np
import pandas as pd

In [3]:
import urllib

In [4]:
import re

In [5]:
import matplotlib.pyplot as plt
import mplleaflet

In [6]:
%matplotlib inline

In [7]:
from readxml import readxml

In [8]:
nm2m=1852. # 1 nautical mile to meters
kt2ms=nm2m/3600.  # knots to m/s

### retrieve from website 

In [9]:
url = 'https://metoc.ndbc.noaa.gov/RSSFeeds-portlet/img/jtwc/jtwc.rss'

In [10]:
print readxml(url,'lxml')

JTWC TROPICAL CYCLONE INFORMATION FEED
https://metoc.ndbc.noaa.gov/web/guest/jtwc
JTWC Tropical Cyclone Information
Thu, 09 Feb 2017 10:54:07 +0000
Thu, 09 Feb 2017 10:54:07 +0000
ContentFeeder 2.0
https://metoc.ndbc.noaa.gov/web/guest/jtwc
JTWC TROPICAL CYCLONE INFORMATION FEED
https://metoc.ndbc.noaa.gov/web/guest/jtwc
Current Northwest Pacific/North Indian Ocean* Tropical Systems
https://metoc.ndbc.noaa.gov/web/guest/jtwc
No Current Tropical Cyclone Warnings.
* Includes Bay of Bengal and Arabian Sea
]]>
CDO.JTWC.fct@navy.mil (JTWC CDO)
Northwest Pacific/North Indian Ocean* Tropical Systems
NWPAC-NIO-WARNINGS
Thu, 09 Feb 17 10:54:07 +0000
Current Central/Eastern Pacific Tropical Systems
https://metoc.ndbc.noaa.gov/web/guest/jtwc
No Current Tropical Cyclone Warnings.
]]>
CDO.JTWC.fct@navy.mil (JTWC CDO)
Central/Eastern Pacific Tropical Systems
EPAC-CPAC-WARNINGS
Thu, 09 Feb 17 10:54:07 +0000
Current Southern Hemisphere Tropical Systems
https://metoc.ndbc.noaa.gov/web/guest/jtwc
Tropic

In [11]:
jtwc=feedparser.parse(url) # parse the webpage

In [12]:
# collect the summeries (one for each alert present in the webpage)
data=jtwc['entries']
for k in range(len(data)):
    if 'tcw' in data[k].summary : txt = (data[k].summary)

In [13]:
elp = [re.sub('<[^<]+>', "", elem) for elem in txt.split('Tropical')] # we split the text to get the references for the alerts

In [14]:
hur = [elem.strip().split('\n')[0] for elem in elp] # the names of the TCs are...

In [15]:
hur

[u'',
 u'Cyclone Formation Alert WTXS21   Cancelled',
 u'Cyclone  04S (Carlos) Warning #10 ']

In [16]:
elp = [re.sub('<[^<]+>', "", elem) for elem in txt.split('href=')] # we now split differently to get the urls for the bulletins

In [17]:
bul = [elem for elem in elp if 'tcw' in elem] # collect the urls 

bul_ =  [elem.strip().split()[0] for elem in bul]

bul_

[u'"https://metoc.ndbc.noaa.gov/ProductFeeds-portlet/img/jtwc/products/sh0417.tcw"']

In [18]:
bul_ = [ elem.encode('utf8').replace('"', '') for elem in bul_] # get rid of unicode characters

In [19]:
bul_

['https://metoc.ndbc.noaa.gov/ProductFeeds-portlet/img/jtwc/products/sh0417.tcw']

## For the first bulletin

In [20]:
try:
    det = urllib.urlopen(bul_[0]).read() # download and parse the bulletin file
except:
    print 'No Bulletins'

In [21]:
print det

WTXS51 PGTW 090300    
WARNING    ATCG MIL 04S SIO 170209012820
2017020900 04S CARLOS     010  01 155 07 SATL 025
T000 236S 0532E 055 R050 020 NE QD 020 SE QD 010 SW QD 000 NW QD R034 070 NE QD 090 SE QD 070 SW QD 050 NW QD 
T012 248S 0541E 060 R050 025 NE QD 025 SE QD 020 SW QD 015 NW QD R034 110 NE QD 100 SE QD 055 SW QD 065 NW QD 
T024 260S 0556E 065 R050 040 NE QD 045 SE QD 025 SW QD 010 NW QD R034 120 NE QD 100 SE QD 050 SW QD 060 NW QD 
T036 275S 0581E 065 R050 060 NE QD 045 SE QD 030 SW QD 020 NW QD R034 150 NE QD 120 SE QD 060 SW QD 065 NW QD 
T048 291S 0607E 060 R050 060 NE QD 050 SE QD 040 SW QD 035 NW QD R034 160 NE QD 130 SE QD 080 SW QD 090 NW QD 
T072 303S 0639E 055 R050 060 NE QD 050 SE QD 040 SW QD 030 NW QD R034 135 NE QD 130 SE QD 120 SW QD 085 NW QD 
T096 309S 0643E 045 R034 085 NE QD 170 SE QD 155 SW QD 095 NW QD 
T120 314S 0637E 035 
AMP
SUBJ:  TROPICAL CYCLONE 04S (CARLOS) WARNING NR 010    
1. TROPICAL CYCLONE 04S (CARLOS) WARNING NR 010    
   01 ACTIVE TROPICAL

In [22]:
#select the first lines where all the info is (conviniently) stored
data =[line.strip() for line in det.splitlines() if ('T' is line.strip()[0]) & ('QD' in line.strip()[-2:])]

In [23]:
data = [re.sub(' ',',',elem) for elem in data] # replace space with commas as delimiter

In [24]:
data = [v.split(',') for v in data] #split to array 

In [25]:
df = pd.DataFrame(data) # create a dataframe

In [26]:
df.head() #check

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,T000,236S,0532E,055,R050,020,NE,QD,020,SE,...,QD,090,SE,QD,070,SW,QD,050,NW,QD
1,T012,248S,0541E,060,R050,025,NE,QD,025,SE,...,QD,100,SE,QD,055,SW,QD,065,NW,QD
2,T024,260S,0556E,065,R050,040,NE,QD,045,SE,...,QD,100,SE,QD,050,SW,QD,060,NW,QD
3,T036,275S,0581E,065,R050,060,NE,QD,045,SE,...,QD,120,SE,QD,060,SW,QD,065,NW,QD
4,T048,291S,0607E,060,R050,060,NE,QD,050,SE,...,QD,130,SE,QD,080,SW,QD,090,NW,QD


In [27]:
pd.concat([df,pd.DataFrame(columns=list(np.arange(df.shape[1],42)))])

,0,1,2,3,4,5,6,7,8,9,...,32,33,34,35,36,37,38,39,40,41
0,T000,236S,0532E,055,R050,020,NE,QD,020,SE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,T012,248S,0541E,060,R050,025,NE,QD,025,SE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,T024,260S,0556E,065,R050,040,NE,QD,045,SE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,T036,275S,0581E,065,R050,060,NE,QD,045,SE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,T048,291S,0607E,060,R050,060,NE,QD,050,SE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,T072,303S,0639E,055,R050,060,NE,QD,050,SE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,T096,309S,0643E,045,R034,085,NE,QD,170,SE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
#expand the dataFrame to include all possible values
if df.shape[1] < 43 : df = pd.concat([df,pd.DataFrame(columns=list(np.arange(df.shape[1],43)))])

In [29]:
df # check

,0,1,2,3,4,5,6,7,8,9,...,33,34,35,36,37,38,39,40,41,42
0,T000,236S,0532E,055,R050,020,NE,QD,020,SE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,T012,248S,0541E,060,R050,025,NE,QD,025,SE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,T024,260S,0556E,065,R050,040,NE,QD,045,SE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,T036,275S,0581E,065,R050,060,NE,QD,045,SE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,T048,291S,0607E,060,R050,060,NE,QD,050,SE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,T072,303S,0639E,055,R050,060,NE,QD,050,SE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,T096,309S,0643E,045,R034,085,NE,QD,170,SE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
# drop the text columns [NE,QD,....]
df = df.drop(df.columns[[6,7,9,10,12,13,15,16,19,20,22,23,25,26,28,29,32,33,35,36,38,39,41,42]], axis=1) 

In [31]:
df = df.set_index(df.columns[0]) # set time as index

In [32]:
df.head() #check

,1,2,3,4,5,8,11,14,17,18,21,24,27,30,31,34,37,40
0,,,,,,,,,,,,,,,,,,
T000,236S,0532E,055,R050,020,020,010,000,R034,070,090,070,050,NaN,NaN,NaN,NaN,NaN
T012,248S,0541E,060,R050,025,025,020,015,R034,110,100,055,065,NaN,NaN,NaN,NaN,NaN
T024,260S,0556E,065,R050,040,045,025,010,R034,120,100,050,060,NaN,NaN,NaN,NaN,NaN
T036,275S,0581E,065,R050,060,045,030,020,R034,150,120,060,065,NaN,NaN,NaN,NaN,NaN
T048,291S,0607E,060,R050,060,050,040,035,R034,160,130,080,090,NaN,NaN,NaN,NaN,NaN


In [33]:
df.columns=np.arange(df.shape[1]) # rename the columns

In [34]:
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,,,,,,,,,,,,,,,,,,
T000,236S,0532E,055,R050,020,020,010,000,R034,070,090,070,050,NaN,NaN,NaN,NaN,NaN
T012,248S,0541E,060,R050,025,025,020,015,R034,110,100,055,065,NaN,NaN,NaN,NaN,NaN
T024,260S,0556E,065,R050,040,045,025,010,R034,120,100,050,060,NaN,NaN,NaN,NaN,NaN
T036,275S,0581E,065,R050,060,045,030,020,R034,150,120,060,065,NaN,NaN,NaN,NaN,NaN
T048,291S,0607E,060,R050,060,050,040,035,R034,160,130,080,090,NaN,NaN,NaN,NaN,NaN


In [35]:
#move the values to appropiate place based on the wind radii in order to force [64,50,34].

for i in range(df.shape[0]):
    if df.iloc[i,3]=='R034' : 
        df.iloc[i,13:] = df.iloc[i,3:8].values
        df.iloc[i,3:8] = None
    elif df.iloc[i,3]=='R050' :
        df.iloc[i,8:] = df.iloc[i,3:13].values
        df.iloc[i,3:8] = None


In [36]:
df #check

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,,,,,,,,,,,,,,,,,,
T000,236S,0532E,055,None,None,None,None,None,R050,020,020,010,000,R034,070,090,070,050
T012,248S,0541E,060,None,None,None,None,None,R050,025,025,020,015,R034,110,100,055,065
T024,260S,0556E,065,None,None,None,None,None,R050,040,045,025,010,R034,120,100,050,060
T036,275S,0581E,065,None,None,None,None,None,R050,060,045,030,020,R034,150,120,060,065
T048,291S,0607E,060,None,None,None,None,None,R050,060,050,040,035,R034,160,130,080,090
T072,303S,0639E,055,None,None,None,None,None,R050,060,050,040,030,R034,135,130,120,085
T096,309S,0643E,045,None,None,None,None,None,None,None,None,None,None,R034,085,170,155,095


In [37]:
df = df.drop(df.columns[[3,8,13]], axis=1) # drop text columns [R064,R050,R034]

In [38]:
#set columns
df.columns=['lat','lon','vmax','64ne','64se','64sw','64nw','50ne','50se','50sw','50nw','34ne','34se','34sw','34nw']

In [39]:
df #check

,lat,lon,vmax,64ne,64se,64sw,64nw,50ne,50se,50sw,50nw,34ne,34se,34sw,34nw
0,,,,,,,,,,,,,,,
T000,236S,0532E,055,None,None,None,None,020,020,010,000,070,090,070,050
T012,248S,0541E,060,None,None,None,None,025,025,020,015,110,100,055,065
T024,260S,0556E,065,None,None,None,None,040,045,025,010,120,100,050,060
T036,275S,0581E,065,None,None,None,None,060,045,030,020,150,120,060,065
T048,291S,0607E,060,None,None,None,None,060,050,040,035,160,130,080,090
T072,303S,0639E,055,None,None,None,None,060,050,040,030,135,130,120,085
T096,309S,0643E,045,None,None,None,None,None,None,None,None,085,170,155,095


In [40]:
tidx = [np.float(elem[1:]) for elem in df.index.values] #convert time from string to float 

In [41]:
df.index=tidx # set float time as index

In [42]:
df # check

,lat,lon,vmax,64ne,64se,64sw,64nw,50ne,50se,50sw,50nw,34ne,34se,34sw,34nw
0.0,236S,0532E,055,None,None,None,None,020,020,010,000,070,090,070,050
12.0,248S,0541E,060,None,None,None,None,025,025,020,015,110,100,055,065
24.0,260S,0556E,065,None,None,None,None,040,045,025,010,120,100,050,060
36.0,275S,0581E,065,None,None,None,None,060,045,030,020,150,120,060,065
48.0,291S,0607E,060,None,None,None,None,060,050,040,035,160,130,080,090
72.0,303S,0639E,055,None,None,None,None,060,050,040,030,135,130,120,085
96.0,309S,0643E,045,None,None,None,None,None,None,None,None,085,170,155,095


In [43]:
df = df.fillna(0) # replace None with zeros

In [44]:
df #check

,lat,lon,vmax,64ne,64se,64sw,64nw,50ne,50se,50sw,50nw,34ne,34se,34sw,34nw
0.0,236S,0532E,055,0,0,0,0,020,020,010,000,070,090,070,050
12.0,248S,0541E,060,0,0,0,0,025,025,020,015,110,100,055,065
24.0,260S,0556E,065,0,0,0,0,040,045,025,010,120,100,050,060
36.0,275S,0581E,065,0,0,0,0,060,045,030,020,150,120,060,065
48.0,291S,0607E,060,0,0,0,0,060,050,040,035,160,130,080,090
72.0,303S,0639E,055,0,0,0,0,060,050,040,030,135,130,120,085
96.0,309S,0643E,045,0,0,0,0,0,0,0,0,085,170,155,095


In [45]:
#create lambda functions for converting the lat lon notation to float
chlat = lambda x: '-'+x[:-1] if x[-1]=='S' else x[:-1]
chlon = lambda x: '-'+x[:-1] if x[-1]=='W' else x[:-1]

In [46]:
# convert lat,lon to -180,180
df.lat = df.lat.map(chlat)
df.lon = df.lon.map(chlon)

In [47]:
df

,lat,lon,vmax,64ne,64se,64sw,64nw,50ne,50se,50sw,50nw,34ne,34se,34sw,34nw
0.0,-236,0532,055,0,0,0,0,020,020,010,000,070,090,070,050
12.0,-248,0541,060,0,0,0,0,025,025,020,015,110,100,055,065
24.0,-260,0556,065,0,0,0,0,040,045,025,010,120,100,050,060
36.0,-275,0581,065,0,0,0,0,060,045,030,020,150,120,060,065
48.0,-291,0607,060,0,0,0,0,060,050,040,035,160,130,080,090
72.0,-303,0639,055,0,0,0,0,060,050,040,030,135,130,120,085
96.0,-309,0643,045,0,0,0,0,0,0,0,0,085,170,155,095


In [48]:
df[['lat','lon']] = df[['lat','lon']].apply(pd.to_numeric,downcast='float').divide(10) #convert to float and divide by 10

In [49]:
df = df.apply(pd.to_numeric,downcast='float') #convert all values to float

In [50]:
df #check

,lat,lon,vmax,64ne,64se,64sw,64nw,50ne,50se,50sw,50nw,34ne,34se,34sw,34nw
0.0,-23.600000,53.200001,55.0,0.0,0.0,0.0,0.0,20.0,20.0,10.0,0.0,70.0,90.0,70.0,50.0
12.0,-24.799999,54.099998,60.0,0.0,0.0,0.0,0.0,25.0,25.0,20.0,15.0,110.0,100.0,55.0,65.0
24.0,-26.000000,55.599998,65.0,0.0,0.0,0.0,0.0,40.0,45.0,25.0,10.0,120.0,100.0,50.0,60.0
36.0,-27.500000,58.099998,65.0,0.0,0.0,0.0,0.0,60.0,45.0,30.0,20.0,150.0,120.0,60.0,65.0
48.0,-29.100000,60.700001,60.0,0.0,0.0,0.0,0.0,60.0,50.0,40.0,35.0,160.0,130.0,80.0,90.0
72.0,-30.299999,63.900002,55.0,0.0,0.0,0.0,0.0,60.0,50.0,40.0,30.0,135.0,130.0,120.0,85.0
96.0,-30.900000,64.300003,45.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,85.0,170.0,155.0,95.0


In [51]:
df.vmax = df.vmax * kt2ms # convert maximum wind speed to m/s

In [52]:
df['notes']=0 ## add column notes for compatibility with operational

In [53]:
#display track
plt.plot(df.lon,df.lat,'o--')
mplleaflet.display()

## output inpData

In [54]:
column_order=['lat','lon','vmax','64ne','64se','64sw','64nw','50ne','50se','50sw','50nw','34ne','34se','34sw','34nw','notes']

In [55]:
header=['lat','long','vmax','64ne','64se','64sw','64nw','50ne','50se','50sw','50nw','34ne','34se','34sw','34nw','notes']

In [56]:
df.to_csv('tmp/inpDataJTWC.txt',index=True, columns=column_order, sep='\t', header=header)